In [1]:
import glob
import time
import gc

#nativos
import os
import unicodedata
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

#terceros
import pandas as pd
import numpy as np
import pymssql
import missingno as msnolistadconoce
from collections import Counter

In [3]:
#terceros
import pandas as pd
import numpy as np
import pymssql

#conneciom MARKET
connection_market = pymssql.connect(
    host='10.11.12.90\BDT',
    user='ibetlmarket',
    port=1433,
    password='m@rk3t2o15',
    database='MARKET'
)

sql_catalogo = """
     SELECT * FROM STG.H_ALERTAS_CATALOGO_ALERTA
"""

catalogo = pd.read_sql(
   sql_catalogo , connection_market
)

connection_market.close()
print(catalogo.shape)
catalogo.head(2)

22

(66, 14)


,COD_ALERTA,COD_VAR_3,FLG_VIGENTE,PESO_FINAL,PESO_CE,CODIGO_BANCA,NOMBRE,BANCA,INFORMACION_LEVANTAR,EXPLIC_VAR1,EXPLIC_VAR2,EXPLIC_VAR3,COD_VAR_1,COD_VAR_2
0,1.0,None,1.0,3.0,3.0,A1,1. El 10% o más del grupo económico del client...,Mediana,No aplica,#Empresas en GE con FEVE Seguir:,Subsector:,None,None,None
1,2.0,None,1.0,2.0,2.0,A2,1. Alguna empresa del grupo económico del clie...,Mediana,No aplica,#Empresas en GE con clas SBS No Normal:,Variación en # de días de rotación de inventar...,None,None,None


In [5]:
catalogo.to_excel("CATALOGO.xlsx", index=False)

In [ ]:
raise Exception("VICTOR DICE: AGUANTA TU COCHE .... valida en el excel y luego corre lo de abajo .... pdta: El excel ya se creó porseaca")

## UP

In [ ]:
catalogo = pd.read_excel('CATALOGO.xlsx', engine='openpyxl')
catalogo.head()

In [ ]:
catalogo.shape

In [ ]:
def generate_orden_banca(val):
    if val <= 23:
        return 'A{}'.format(val)
    if val <= 34:
        return 'A{}'.format(val - 23)
    if val <= 45:
        return 'A{}'.format(val - 34)
    if val <= 66:
        return 'A{}'.format(val - 45)
    
    return 'X'

In [ ]:
catalogo['CODIGO_BANCA'] = catalogo['COD_ALERTA'].apply(generate_orden_banca)

In [ ]:
columns = list(catalogo.columns)
agregar_antes = columns.pop()
print(columns)

In [ ]:
columns.insert(1, agregar_antes)

In [ ]:
print(columns)

In [ ]:
catalogo = catalogo[columns]
catalogo.head()

In [ ]:
%%time
# mssql+pymssql
from sqlalchemy import create_engine
engine = create_engine(r"mssql+pymssql://ibetlmarket:m@rk3t2o15@10.11.12.90\BDT/MARKET")
connection_x = engine.connect()

try:
    catalogo.to_sql(
        name='H_ALERTAS_CATALOGO_ALERTA',
        schema='STG',
        con=connection_x, 
        index=False, 
        if_exists='replace'
    )
    connection_x.close()
    correct = 1

except Exception as e:
    print("ERROR: ", str(e))
    correct = 0

print(correct)

In [ ]:
import pymssql
print("Inciando actualizacion de tblas ODS")
connection = pymssql.connect(
    host='10.11.12.90\BDT',
    user='ibetlmarket',
    port=1433,
    password='m@rk3t2o15',
    database='MARKET'
)

In [ ]:
catalogo.columns

In [ ]:
with connection.cursor() as cursor:
    try:
        print("/"*50)
        alter_text_files = """
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN COD_ALERTA FLOAT;
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN CODIGO_BANCA VARCHAR(20);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN NOMBRE VARCHAR(512);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN BANCA VARCHAR(512);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN EXPLIC_VAR1 VARCHAR(512);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN EXPLIC_VAR2 VARCHAR(512);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN EXPLIC_VAR3 VARCHAR(512);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN COD_VAR_1 VARCHAR(255);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN COD_VAR_2 VARCHAR(255);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN COD_VAR_3 VARCHAR(255);
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN FLG_VIGENTE FLOAT;
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN PESO_FINAL FLOAT;
        ALTER TABLE STG.H_ALERTAS_CATALOGO_ALERTA ALTER COLUMN PESO_CE FLOAT;
        """
        cursor.execute(alter_text_files)
    except Exception as e:
        print("ERROR: ", str(e))

try:
    connection.commit()
    connection.close()
    print("Conexion database cerrada")
except Exception as e:
    print("Error:  ", str(e))